In [1]:
from cblearn import datasets
import numpy as np
import cblearn
import tqdm
from scipy.sparse import csr_matrix,coo_matrix
import time
import sklearn
import sys
sys.path.append('../..')
from comparisonHC import ComparisonHC,tSTE

In [2]:
#fetching car dataset
data = datasets.fetch_car_similarity()

In [3]:
data

{'triplet': array([[ 9,  8, 39],
        [44, 43, 41],
        [38, 22,  0],
        ...,
        [29, 28, 31],
        [ 2,  5, 29],
        [ 2, 14,  1]]),
 'response': array([1, 1, 2, ..., 2, 1, 2], dtype=int64),
 'rt_ms': array([[6.918],
        [4.832],
        [8.596],
        ...,
        [3.361],
        [2.232],
        [3.888]]),
 'class_id': array([0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 3, 1, 1, 0, 3, 3, 2, 0,
        2, 0, 1, 0, 0, 3, 0, 2, 0, 1, 0, 0, 1, 1, 1, 1, 1, 3, 1, 3, 0, 1,
        1, 1, 3, 3, 3, 3, 3, 3, 2, 3, 1, 3, 0, 1, 3, 1]),
 'class_name': array(['OFF-ROAD / SPORT UTILITY VEHICLES', 'ORDINARY CARS', 'OUTLIERS',
        'SPORTS CARS'], dtype='<U33'),
 'DESCR': '.. _central_car_dataset:\n\nCar Similarity dataset\n-----------------------\n\n`This dataset contains triplets`_ of 60 car images, responsed in an online survey.\nThe people chose one car of three, such that the following statement is true:\n"Object A is the most central object within the triple of ob

In [4]:
#getting raw triplets from car data
triplets_raw = data['triplet']

In [5]:
n = np.amax(triplets_raw) + 1

In [6]:
#objects in the data
n

60

In [7]:
n_triplets_raw = triplets_raw.shape[0]

In [8]:
#raw triplets in the data(number of responses)
n_triplets_raw

7097

In [9]:
#getting standard triplets from raw triplets. Random triplets are of form (i,j,k) where s_ij>s_ik
triplets = cblearn.preprocessing.triplets_from_mostcentral(data['triplet'],data['response'])

In [10]:
n_triplets = triplets.shape[0]

In [11]:
#standard triplets obtained from the data
n_triplets

14194

In [12]:
n_triplets_raw

7097

In [13]:
def get_k(i,j,n_examples):
    """Given the row and column coordinates, returns the index of entries
            of an off-diagonal upper triangular matrix where the elements
            are taken in a row-major order:
            [. 0 1 2 3
             . . 4 5 6
             . . . 7 8
             . . . . 9
             . . . . .]
            Parameters
            -------
            i : int or numpy array
                The row index of the example, between 0 and n_examples.
            j : int or numpy array, shape(i.shape)
                The column index of the example, between i and n_examples.
            n_example : int
                The number of rows and columns in the matrix. If None,
                self.n_examples is used. (Default: None).
            Returns
            ----------
            k : int or numpy array, shape(i.shape)
                The row-major index of the example, between 0 and (n_examples choose 2).
            Notes
            -----
            The original formulation was taken from the following link:
            https://stackoverflow.com/questions/27086195/linear-index-upper-triangular-matrix
    """
    k = ((n_examples*(n_examples-1))//2) - ((n_examples-i)*((n_examples-i)-1))//2 + j - i - 1

    return k

In [14]:
def get_ij(k,n_examples):
        """Returns the row and column coordinates given the index of the
        entries of an off-diagonal upper triangular matrix where the
        elements are taken in a row-major order:
        [. 0 1 2 3
         . . 4 5 6
         . . . 7 8
         . . . . 9
         . . . . .]
        Parameters
        ----------
        k : int or numpy array
            The row-major index of the example, between 0 and
            (n_examples choose 2).
       
        n_example : int
            The number of rows and columns in the matrix. If None,
            self.n_examples is used. (Default: None).
        Returns
        -------
        i : int or numpy array, shape(k.shape)
            The row index of the example, between 0 and n_examples.
        j : int or numpy array, shape(k.shape)
            The column index of the example, between i and n_examples.
        Notes
        -----
        The original formulation was taken from the following link:
        https://stackoverflow.com/questions/27086195/linear-index-upper-triangular-matrix
        """

        i = n_examples - 2 - (np.sqrt(-8*k + 4*n_examples*(n_examples-1)-7)/2 - 1/2).astype(int)
        j = k + i + 1 - (n_examples*(n_examples-1))//2 + ((n_examples-i)*((n_examples-i)-1))//2
        
        return i,j

In [15]:
def get_AddS_comparisons(comparisons,n_examples):
        """Get a sparse matrix representing the comparisons in a way that is
        easy to handle for AddS.
        Returns
        -------
        AddS_comparisons : scipy csr matrix, shape(n_examples**2,n_examples)
            A scipy csr_matrix containing values in {1,-1,0}. Given
            i!=j,k, in entry (i*n_examples+j,k), the value 1 indicates
            that the triplet (i,j,k) is available, the value -1
            indicates that the triplet (i,k,j) is available, and the
            value 0 indicates that neither of the triplets is
            available.
        """
        i = comparisons.row
        j,k = get_ij(comparisons.col,n_examples)

        n_pairs = n_examples
        
        rows_j = i*n_pairs+j
        columns_j = k
               
        rows_k = i*n_pairs+k
        columns_k = j
               
        rows = np.concatenate((rows_j,rows_k))
        columns = np.concatenate((columns_j,columns_k))
        entries = np.concatenate((comparisons.data,-comparisons.data))
            
        AddS_comparisons = csr_matrix((entries,(rows,columns)),shape=(n_examples**2,n_examples),dtype=int)
                
        return AddS_comparisons

In [16]:
def get_AddS_triplets(comparisons,n_examples):
    """Returns a symmetric similarity matrix representing the similarities
    between all the examples using the AddS triplets approach.
    Parameters
    ----------
    comparisons : scipy csr matrix 
        It is a scipy csr matrix,
        shape(n_examples**2,n_examples) A scipy csr_matrix containing
        values in {1,-1,0}. Given i!=j,k, in entry (i*n_examples+j,k),
        the value 1 indicates that the triplet (i,j,k) is available,
        the value -1 indicates that the triplet (i,k,j) is available,
        and the value 0 indicates that neither of the triplets is
        available.
    n_examples : int
        The number of examples.
    Returns
    -------
    kernel : numpy array, shape (n_examples,n_examples)
        A nummpy array of similarities between the examples.
    """
    
    kernel = np.zeros((n_examples,n_examples))
    
    comps = comparisons
    entries = comps.sum(axis=1).A1

    indices = np.arange(n_examples**2)
    i = indices//n_examples
    j = indices - i*n_examples
    
    kernel[i,j] = entries
    
    kernel += kernel.transpose()

    return kernel

In [17]:
def get_MulK_comparisons(comparisons):
    """Get a sparse matrix representing the comparisons in a way that is
      easy to handle for MulK.
      Returns
      -------
      MulK_comparisons : scipy csr matrix, shape(n_examples,(n_examples choose 2)) 
          A scipy csr_matrix containing values in {1,-1,0}. Given
          i!=j,k, j<k, in entry (i,self._get_k(j,k)), the value 1
          indicates that the triplet (i,j,k) is available, the value
          -1 indicates that the triplet (i,k,j) is available, and
          the value 0 indicates that neither of the triplets is
          available.
      """
    MulK_comparisons = comparisons.tocsr()
    
    return MulK_comparisons

In [18]:
def get_MulK_triplets(comparisons,n_examples):
    """Returns a symmetric similarity matrix representing the similarities
    between all the examples using the MulK triplets approach.
    Parameters
    ----------
    comparisons : scipy csr matrix
        It is a scipy csr matrix
        of shape(n_examples,(n_examples choose 2)) containing values
        in {1,-1,0}. Given i!=j,k, j<k, in entry (i,self._get_k(j,k)),
        the value 1 indicates that the triplet (i,j,k) is available,
        the value -1 indicates that the triplet (i,k,j) is available,
        and the value 0 indicates that neither of the triplets is
        available.
    n_examples : int
        The number of examples handled.
    Returns
    -------
    kernel : numpy array, shape (n_examples,n_examples)
        A nummpy array of similarities between the examples.
    """
    kernel = np.zeros((n_examples,n_examples))
    
    kernel = comparisons.dot(comparisons.transpose())
    
    norms = np.sqrt(comparisons.getnnz(axis=1))
    norms = norms.reshape(-1,1) @ norms.reshape(1,-1)
    norms = np.where(norms == 0,1,norms) # This is to avoid issues with the true divide when the norm is 0 for i or j
    
    kernel = kernel.toarray()/norms
    np.fill_diagonal(kernel,0)

    return kernel

In [19]:
def get_tSTE_triplets(triplets,n,n_triplets):
    """Returns a symmetric similarity matrix representing the similarities
    between all the examples using the MulK triplets approach.
    Parameters
    ----------
    triplets : standard triplets
    
    n : int
        The number of objects.
        
    n_trilets : int
        The number of triplets.
    Returns
    -------
    kernel : numpy array, shape (n_examples,n_examples)
        A nummpy array of similarities between the examples.
    """
    
    X = tSTE(triplets,n,n_triplets)
    kernel = sklearn.metrics.pairwise.cosine_similarity(X, dense_output=True)
    
    return kernel

In [20]:
#getting rows, columns and entries to represent the triplets as an upper triangular matrix
rows = []
columns = []
entries = []
for triplet in triplets:
    rows.append(triplet[0])
    if (triplet[1]<triplet[2]):
        columns.append(get_k(triplet[1],triplet[2],n))
        entries.append(1)
    else:
        columns.append(get_k(triplet[2],triplet[1],n))
        entries.append(-1)

In [21]:
rows = np.array(rows)
columns = np.array(columns)
entries = np.array(entries)

In [22]:
n_entries = (n*(n-1))//2

In [23]:
n_entries

1770

In [24]:
#getting a sparse matrix
comparisons = coo_matrix((entries,(rows,columns)),shape=(n,(n*(n-1))//2),dtype=int)
comparisons.eliminate_zeros()

In [25]:
comp = get_AddS_comparisons(comparisons,n)

In [26]:
adds_similarities= get_AddS_triplets(comp,n)

In [27]:
#running comparison hierarchical clustering using AddS-3 similarities
chc = ComparisonHC(adds_similarities,n)
chc.fit([[j] for j in range(n)])
print("ComparisonHC ran for {:.2f} seconds.".format(chc.time_elapsed))
print("Revenue with AddS-3: {}".format(-chc.cost_dasgupta(adds_similarities)))

ComparisonHC ran for 0.67 seconds.
Revenue with AddS-3: 152116.0


In [28]:
comp_mulk = get_MulK_comparisons(comparisons)

In [29]:
mulk_similarities = get_MulK_triplets(comp_mulk,n)

In [30]:
#running comparison hierarchical clustering using MulK-3 similarities
chc_mulk = ComparisonHC(mulk_similarities,n)
chc_mulk.fit([[j] for j in range(n)])
print("ComparisonHC ran for {:.2f} seconds.".format(chc_mulk.time_elapsed))
print("Revenue with Mulk-3: {}".format(-chc_mulk.cost_dasgupta(adds_similarities)))

ComparisonHC ran for 0.66 seconds.
Revenue with Mulk-3: 126417.0


In [31]:
#running comparison hierarchical clustering using tSTE similarities
#since it is a randomised process it is run 10 times and mean and standard deviation are obtained
tste_cost = []
total_time = 0.0
for i in tqdm.tqdm(range(10)):
    tste_similarities = get_tSTE_triplets(triplets,n,n_triplets)
    chc_tste = ComparisonHC(tste_similarities,n)
    chc_tste.fit([[j] for j in range(n)])
    total_time = total_time+chc_tste.time_elapsed
    tste_cost.append(-chc_tste.cost_dasgupta(adds_similarities))
print("ComparisonHC ran for {:.2f} seconds.".format(total_time))

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:45<00:00,  4.52s/it]

ComparisonHC ran for 6.23 seconds.


In [32]:
tste_cost = np.array(tste_cost)
mean = np.mean(tste_cost)
std = np.std(tste_cost)

In [33]:
print("Mean revenue obtained with tSTE after 10 runs: {}".format(mean))

Mean revenue obtained with tSTE after 10 runs: 155245.7


In [34]:
print("Standard deviation of revenue obtained with tSTE after 10 runs: {}".format(std))

Standard deviation of revenue obtained with tSTE after 10 runs: 1871.1542988219865
